# Econometrics II: Homework 5
## Eric Schulman
## November 15, 2018

In [1]:
import pandas as pd
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

### Part 1 - Part 4

In [2]:
sigma = .1
var = [[1, sigma ], [sigma , 1]]
mean = [0., 0.]
N = int(1e6)

ey, ex = np.random.multivariate_normal(mean, var, size = N).transpose()

In [3]:
pz = .3

z = np.random.binomial(1,pz,N).astype(np.float)

In [4]:
beta_0x = -.5
beta_1x = 2

x = (beta_0x + beta_1x*z + ex > 0).astype(np.float) 

In [5]:
beta_0y = 1
beta_1y = 1

y = beta_0y + beta_1y*x + ey

### Part 5

$$E(y(1) -y(0)) = $$
$$E( (\beta_{0y} + \beta_{1jy} + \epsilon_{yj}) - (\beta_{0y} + \epsilon_{yj}) ) =$$
$$E(\beta_{1jy}) = \beta_{1y} = 1$$

### Part 6

* What happens to the OLS estimator applied to this data?

OLS is biased upward.

* Is that expected or unexpected?

This is expected since treatment is endogenously determined.

In [6]:
model = sm.OLS(y, sm.add_constant(x))
model = model.fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                 3.124e+05
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:22:22   Log-Likelihood:            -1.4162e+06
No. Observations:             1000000   AIC:                         2.832e+06
Df Residuals:                  999998   BIC:                         2.832e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9447      0.001    672.935      0.0

### Part 7

Using the IV estimator recovers the average treatment effect.

In [7]:
model =  IV2SLS(y, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.236
Model:                         IV2SLS   Adj. R-squared:                  0.236
Method:                     Two Stage   F-statistic:                 8.245e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Wed, 14 Nov 2018                                         
Time:                        14:22:24                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0006      0.002    501.236      0.0

### Part 8 


* They are **compliers**:
    
    If $x_i = 1, z_i = 1$ and $x_i = 0, z_i = 0$ 
    
    Thus, $-\beta_{0x} > \epsilon_{xj} > -\beta_{0x} - \beta_{1x}$

*  They are **defiers**:
    
    If $x_i = 0, z_i = 1$ and $x_i = 1, z_i = 0$  
    
    Thus, $\epsilon_{xj} \leq -\beta_{0x} - 
\beta_{1x}$ and $\epsilon_{xj} > - \beta_{1x}$ 

* They are never **takers**:
    
    If $x_i = 0, z_i = 0$ and $x_i = 0, z_i = 1$ 
    
    Thus, $\epsilon_{xj} > -\beta_{0x} $

* They are **always takers**:
    
    If $x_i = 1, z_i = 0$ and $x_i = 1, z_i = 0$
    
    Thus, $\epsilon_{xj} \leq -\beta_{0x} - \beta_{1x}$


### Part 9 

Above we can see that defiers have $\epsilon_{xj} \leq -\beta_{0x} - \beta_{1x}$ and $\epsilon_{xj} > - \beta_{1x}$ 

When   $\beta_{0x} < \beta_{0x} + \beta_{1x}$ we have no defiers because this set is empty. This is essentially a monotone treatment selection assumptions. Essentially, getting the insturment makes you more likely to get treated.

### Part 10

In [8]:
c = ( (- beta_0x > ex) * (ex > - beta_0x -  beta_1x) ).astype(np.float) 
print 'compliers: %s'% (c.sum()/N)

d = ( (- beta_0x  < ex) * (ex < - beta_0x -  beta_1x) ).astype(np.float) 
print 'defiers: %s'%(d.sum()/N)

a =   ( (- beta_0x < ex)  ).astype(np.float) 
print 'always takers: %s'%(a.sum()/N)

n= ( (- beta_0x -  beta_1x > ex)  ).astype(np.float) 
print 'never takers: %s'%(n.sum()/N)


compliers: 0.624664
defiers: 0.0
always takers: 0.308394
never takers: 0.066942


### Part 11

In [9]:
x1c = x*c #x is 1 and a complier
x0c = (1-x)*c #x is 0 and a complier


y1c = (y*x1c).sum()/x1c.sum()
y0c =  (y*x0c).sum()/x0c.sum()

print 'LATE: %s'%(y1c - y0c)

LATE: 1.0001576378184347


### Part 12 

 The IV estimate is the LATE. The OLS estimator is not LATE.

### Part 13

Since $\beta_{1jy} =1$ for all $j$, the treatment effect and the local average treatment effect should be the same. Everyone has the same treatment effect.

### Part 14

In [10]:
beta_1jy = np.random.normal(1, 1, size = N)

yj = beta_0y + beta_1jy*x + ey

### Part 15

$$E(y(1) -y(0)) = $$
$$E( (\beta_{0y} + \beta_{1jy} + \epsilon_{yj}) - (\beta_{0y} + \epsilon_{yj}) ) =$$
$$E(\beta_{1jy}) = 1$$


### Part 16

It does not. The individual coefficient is independent from selection process. However, selection is still endogenously determined

In [11]:
model = sm.OLS(yj, sm.add_constant(x))
model = model.fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                 2.080e+05
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:22:29   Log-Likelihood:            -1.6185e+06
No. Observations:             1000000   AIC:                         3.237e+06
Df Residuals:                  999998   BIC:                         3.237e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9447      0.002    549.669      0.0

### Part 17

The IV estimator does recover the average treatment effect from question 15.

In [12]:
model =  IV2SLS(yj, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.171
Model:                         IV2SLS   Adj. R-squared:                  0.171
Method:                     Two Stage   F-statistic:                 5.567e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Wed, 14 Nov 2018                                         
Time:                        14:22:30                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9974      0.002    408.377      0.0

### Part 18

They do not. The coefficient is random, but it does not depend on $\epsilon_x$ or $\epsilon_y$. Since selection into the treatment group is independent from the response to treatment, nothing changes.

### Part 19

In [13]:
yj1c = (yj*x1c).sum()/x1c.sum()
yj0c =  (yj*x0c).sum()/x0c.sum()

print yj1c - yj0c

1.0019748933596002


### Part 20
 19 and 17 are roughly the same. 15 is biased upward due to selection
 
### Part 21
The local average treatment effect is the average treatment effect. This is because the average treatment effect is the same among all groups.
### Part 22

In [14]:
beta_1iy = np.random.normal(1 + ex, 1, size = N)

yi = beta_0y + beta_1iy*x + ey

### Part 23
The average treatment effect increases. Because selection affects the treatment effect.

$$E(y(1) -y(0)) = $$

$$E( \beta_{1jy}(1) - \beta_{1jy}(0) )  = $$

$$E( \epsilon_{xj}(1) - \epsilon_{xj}(0) ) =  $$


$$E(\epsilon_{xj}(1) |z_j =1 )Pr(z_j = 1) + E( \epsilon_{xj}(1) | z_j = 0 )Pr(z_j = 0)  - E(\epsilon_{xj}(0) | z_j=1)Pr(z_j=1) - E(\epsilon_{xj}(0) | z_j=0)Pr(z_j=0)  = $$

$$ [ E(\epsilon_{xj}(1) |z_j =1 )Pr(z_j = 1) - E(\epsilon_{xj}(0) | z_j=1) ]Pr(z_j=1) + [E( \epsilon_{xj}(1) | z_j = 0 ) -  E(\epsilon_{xj}(0) | z_j=0)]Pr(z_j=0)  $$

Focusing on one of these terms one by one:

$$ E( \beta_{1jy}(1) | z_j = 0 ) = E(\epsilon_{xj} | \epsilon_{xj} > -\beta_{1x} ) \approx 1.141 $$

$$ E(\beta_{1jy}(0) |  z_j = 1 )  =  \rho E(\epsilon_{xj} | \epsilon_{xj} < -\beta_{0x} -\beta_{1x} )  \approx -1.939 $$

$$ E( \beta_{1jy}(1) | z_j = 1 ) =  \rho E(\epsilon_{xj} | \epsilon_{xj} > -\beta_{0x} -\beta_{1x} )  \approx .139 $$

$$ E(\beta_{1jy}(0) |  z_j = 0 )   =  \rho E(\epsilon_{xj} | \epsilon_{xj} < -\beta_{1x} )  \approx -.509 $$



Plugging in these terms we get, 

$$E(y(1) -y(0)) \approx 1.779$$

### Part 24
Below we calculate the OLS estimator based on the numerical simulation. It does not recover the average treatment effect. We are biased downward

In [15]:
model = sm.OLS(yi, sm.add_constant(x))
model = model.fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                 3.640e+05
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:22:37   Log-Likelihood:            -1.7577e+06
No. Observations:             1000000   AIC:                         3.515e+06
Df Residuals:                  999998   BIC:                         3.515e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9447      0.002    478.275      0.0

### Part 25

Because of the heterogeneity, we do not recover the average treatment effect

In [16]:
model =  IV2SLS(yi, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.165
Model:                         IV2SLS   Adj. R-squared:                  0.165
Method:                     Two Stage   F-statistic:                 1.526e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Wed, 14 Nov 2018                                         
Time:                        14:22:40                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4632      0.003    488.868      0.0

### Part 26

The average treatment effect increases. The IV estimate decreases. Neither actually recover the average treatment effect. The IV estimate is just the LATE.

### Part 27

The LATE is estimated by the IV as we can see below

In [17]:
yi1c = (yi*x1c).sum()/x1c.sum()
yi0c =  (yi*x0c).sum()/x0c.sum()

print yi1c - yi0c

0.6447214147103048


### Part 28

The IV estimate is the same as the LATE. It is different from the OLS estimate though